In [1]:
import pandas
import gripql
conn = gripql.Connection("http://grip.compbio.ohsu.edu")
O = conn.graph("bmeg_rc1_2")

In [18]:
q = O.query().V("Project:TCGA-KIRC")
q = q.in_("InProject").in_("SampleFor").has(gripql.eq("gdc_attributes.sample_type", "Primary Tumor"))
q = q.in_("AliquotFor").in_("CallsetFor").outE("AlleleCall")
q = q.has(gripql.contains("methods", "MUTECT"))
q = q.aggregate(gripql.term("geneCount", "ensembl_gene"))

res = list(q)
counts = {}
for i in res[0]['geneCount']['buckets']:
    counts[i['key']] = i['value']


[INFO]	2019-03-01 17:22:30,156	1 results received in 6 seconds
